In [1]:
import os

# đọc video từ thư mục data/video
video_path = "data/video/"
video_list = os.listdir(video_path)

In [2]:
# thêm vào danh sách video các video lật ngược name+f
new_video_list = video_list.copy()

for video in video_list:
    new_video_list.append(video[:-4] + "f.mp4")

video_list = new_video_list

In [3]:
import cv2
import sys

sys.path.append("../module/")
from init import init_csv
from pose_landmarker import extract_pose_features

list = []
for video_name in video_list:
    # Nếu tên video có chữ f ở cuối thì xóa chữ f đi và lật ngược video
    if video_name[-5] == "f":
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name[:-5] + ".mp4"))
    else:
        # Đọc video
        cap = cv2.VideoCapture(os.path.join(video_path, video_name))

    # Kiểm tra xem video có được mở thành công không
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()

    # Khởi tạo csv_writer
    csv_writer = init_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Đọc từng frame từ video
    while True:
        ret, frame = cap.read()

        # Kiểm tra nếu có chữ f ở cuối tên video thì lật ngược frame
        if video_name[-5] == "f":
            frame = cv2.flip(frame, 1)

        # Kiểm tra xem có còn frame nào để đọc không
        if not ret:
            break

        # Trích xuất đặc trưng từ frame
        landmarks = extract_pose_features(frame)

        if landmarks is not None:
            # Chuyển landmarks thành mảng 1 chiều
            list = [j for i in landmarks for j in i]
            # Ghi dữ liệu xuống file csv
            csv_writer.writerow(list)

        # Thời gian chờ giữa các frame (để đảm bảo đọc video với tốc độ 30 fps)
        wait_time = int(1000 / 30)

# Đóng cửa sổ hiển thị và giải phóng tài nguyên
cv2.destroyAllWindows()

In [4]:
import pandas as pd
import csv

# Khởi tạo csv_writer
csv_file = open("data/data.csv", "w", newline="")  # Tạo file csv
csv_writer = csv.writer(csv_file)  # Khởi tạo csv_writer

# Khởi tạo header cho file csv
header = [f"{i}" for i in range(1, 337)] + ["label"]
csv_writer.writerow(header)

for video_name in video_list:
    # Đọc file csv
    csv_reader = pd.read_csv(f"data/csv/{video_name.split('.')[0]}.csv")

    # Lấy 28 dòng đầu tiên của file csv và chuyển nó thành mảng 1 chiều
    list = [j for i in csv_reader.head(28).values for j in i]

    # Nếu tên video có số 0 đầu thì thêm số 0 và cuối list
    if video_name.split(".")[0][0] == "0":
        list.append(0)
    else:
        list.append(1)

    # Ghi dữ liệu xuống file csv
    csv_writer.writerow(list)

# Đóng file csv
csv_file.close()

In [5]:
# Đọc file data.csv
csv_reader = pd.read_csv("data/data.csv")

# In ra 5 dòng đầu tiên của file csv
csv_reader

,1,2,3,4,5,6,7,8,9,10,...,328,329,330,331,332,333,334,335,336,label
0,0.482556,0.301600,-0.016666,0.478341,0.489260,-0.000020,0.519976,0.788471,0.122996,0.430674,...,0.474062,0.483298,0.000002,0.516464,0.785509,0.101732,0.431066,0.781687,0.098530,0
1,0.474785,0.292848,-0.047951,0.480258,0.498806,-0.000009,0.548909,0.791485,0.136930,0.465594,...,0.482976,0.486595,0.000028,0.498083,0.799704,0.090453,0.418309,0.778476,0.100237,0
2,0.491824,0.284425,0.014305,0.482035,0.483415,0.000027,0.497900,0.798342,0.084356,0.444572,...,0.477217,0.492717,0.000066,0.497273,0.792012,0.143285,0.459336,0.794777,0.134134,0
3,0.477825,0.290468,-0.051541,0.479761,0.494881,-0.000031,0.549031,0.790939,0.080075,0.462719,...,0.486793,0.495827,-0.000026,0.500054,0.796067,0.144824,0.396622,0.784166,0.043748,0
4,0.484249,0.292227,-0.006678,0.484738,0.495959,0.000036,0.499913,0.796029,0.119156,0.462000,...,0.480667,0.495396,0.000031,0.500835,0.794080,0.122698,0.462593,0.793039,0.126805,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.653652,0.509729,0.000099,0.809156,0.499647,-0.037694,0.808862,0.526558,-0.005777,1
183,0.540305,0.497453,-0.267193,0.650201,0.519780,0.000115,0.810189,0.501984,-0.184165,0.810563,...,0.578643,0.523901,-0.000021,0.685590,0.484553,-0.038132,0.700308,0.492732,-0.182590,1
184,0.487860,0.541461,-0.444910,0.531358,0.526937,0.000063,0.491650,0.550740,0.252951,0.506616,...,0.579306,0.577449,-0.000180,0.610150,0.555996,0.229243,0.605048,0.562813,-0.166991,1
